# Plot Trajectories
This is script will create a sample plot of trajectories calculated from "backwards_trajectories.ipynb". It contains two subplots: trajectories in the x-y and x-z plane.  

# Load Modules

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os, sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.collections import LineCollection
#from pathos.multiprocessing import ProcessingPool as Pool
from multiprocessing import Pool

sys.path.append('/uufs/chpc.utah.edu/common/home/u1013082/lake_effect/modules')
import nclcmaps

# Read in Trajectory Data

In [2]:
xpos = np.load('/uufs/chpc.utah.edu/common/home/u1013082/lake_effect/cm1/trajectory_data/xpos_2000m_tug_smooth.npy')
ypos = np.load('/uufs/chpc.utah.edu/common/home/u1013082/lake_effect/cm1/trajectory_data/ypos_2000m_tug_smooth.npy')
zpos = np.load('/uufs/chpc.utah.edu/common/home/u1013082/lake_effect/cm1/trajectory_data/zpos_2000m_tug_smooth.npy')
variable1 = np.load('/uufs/chpc.utah.edu/common/home/u1013082/lake_effect/cm1/trajectory_data/th_2000m_tug_smooth.npy')

# Read in Model Data for Domain Info

In [3]:
#Read in model data
ds = xr.open_dataset('/uufs/chpc.utah.edu/common/home/steenburgh-group8/tom/cm1/output/12ms_2000m_tug.nc', chunks={'time': 5})
#Horizontal resolution of model output (meters)
hor_resolution = (ds.xf[1].values-ds.xf[0].values)*1000

#Get surface height grid (set to zero if no terrain)
try:
    zs = np.array(ds.zs[0,:,:])
except:
    zs = np.zeros((ds.ny, ds.nx))  

# Create Cluster

In [4]:
from dask.distributed import Client, LocalCluster
c = LocalCluster(n_workers = 40, threads_per_worker=1)
client = Client(c)

# Model Info

In [5]:
#Plot Dimensions
top = 41
left = 0

ymid = np.int(ds.ny/2)
x1d = ds.xh[left:].values
y1d = ds.yh.values
x2d = np.rot90(np.repeat(x1d,top, axis = 0).reshape(ds.nx-left,top))
y2d = np.rot90(np.repeat(y1d,top, axis = 0).reshape(ds.ny-left,top))
#z2dxz = np.repeat(ds.z[:top].values*1000,ds.nx-left, axis = 0).reshape(top, ds.nx-left)
z2dyz = np.repeat(ds.z[:top].values*1000,ds.ny-left, axis = 0).reshape(top, ds.ny-left)
z2dxz = ds.zh[0,:top,ymid,left:].values

#Model output
ts = 0
te = 500

#qcxy = ds.qc[time,:top,:,:]
#qrxy = ds.qr[time,:top,:,:]
#qixy = ds.qi[time,:top,:,:]
#qgxy = ds.qg[time,:top,:,:]
#qsxy = ds.qs[time,:top,:,:]
#Total
#qxy  = (qcxy + qrxy + qixy + qgxy + qsxy).max(dim = ('nk')).values*1000
qxy  = ds.cref[ts:te,:,:].values


#qcxz = ds.qc[time,:top,:,:]
#qrxz = ds.qr[time,:top,:,:]
#qixz = ds.qi[time,:top,:,:]
#qgxz = ds.qg[time,:top,:,:]
#qsxz = ds.qs[time,:top,:,:]
#Total
#qxz  = (qcxz + qrxz + qixz + qgxz + qsxz).max(dim = ('nj')).values*1000
qxz  = ds.dbz[ts:te,:top,ymid,:].values



#qcyz = ds.qc[time,:top,:,:]
#qryz = ds.qr[time,:top,:,:]
#qiyz = ds.qi[time,:top,:,:]
#qgyz = ds.qg[time,:top,:,:]
#qsyz = ds.qs[time,:top,:,:]
#Total
#qyz  = (qcyz + qryz + qiyz + qgyz + qsyz).max(dim = ('ni')).values*1000
qyz  = ds.dbz[ts:te,:top,:,2800].values



# Plot Trajectories

In [6]:
###############################################################################
############################ Trajectory Plot  #################################
###############################################################################
#%matplotlib inline

#Loop over all time steps
for t in np.arange(0,496,1):
#def plotting(t):
#t = 20

    fig = plt.figure(figsize=(15*0.75,9.5*0.75), facecolor='w', edgecolor='k')
    plt.subplots_adjust(left=0.07, bottom=0.1, right=0.995, top=0.98, wspace=0.8, hspace=0.675)

    #Color range of Varible1 (set by user)
    xlmin = 266
    xlmax = 275.01
    xlevels = np.arange(xlmin,xlmax, 0.05)
    xlevels_ticks = np.arange(xlmin,xlmax,2)
    xlevels_ticks_labels = np.arange(xlmin,xlmax, 2).astype(int)

    #Levels for qc
    lmin = 5
    lmax = 41
    levels = (np.arange(lmin,lmax, 0.5))
    levels_ticks = np.arange(lmin,lmax,5)
    levels_ticks_labels = np.round(levels_ticks, 5)


    #Colormaps
    cmap = cm.Spectral_r
    cmap = cm.viridis

    #Colormap
    colors1 = np.array(nclcmaps.colors['amwg_blueyellowred'])
    colors_int = colors1.astype(int)
    colors = list(colors_int)
    cmap = nclcmaps.make_cmap(colors, bit=True)

    #Colormap
    colors1 = np.array(nclcmaps.colors['prcp_1'])
    colors_int = colors1.astype(int)
    colors = list(colors_int)
    cmapf = nclcmaps.make_cmap(colors, bit=True)
    #cmapf = cm.Greys
    alpha = 0.5
    width = 2.4
    TF = True
    alpha_traj = 1
    alpah_barb = 0.5

    #Dimensions
    left = 1500
    right = 3600
    y_top = 1000
    y_bot = 0
    size = 6

    ############  Functions for coloring lines with third variable ################

    def make_segments(x, y):
        '''
        Create list of line segments from x and y coordinates, in the correct format for LineCollection:
        an array of the form   numlines x (points per line) x 2 (x and y) array
        '''
        points = np.array([x, y]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        return segments

    # Interface to LineCollection:

    def colorline(x, y, z=None, cmap = cm.Spectral_r, norm=matplotlib.colors.BoundaryNorm(xlevels,cmap.N), linewidth=2, alpha=0.3):
        '''
        Plot a colored line with coordinates x and y
        Optionally specify colors in the array z
        Optionally specify a colormap, a norm function and a line width
        '''
        # Default colors equally spaced on [0,1]:
        if z is None:
            z = np.linspace(0.0, 1.0, len(x))     
        # Special case if a single number:
        if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
            z = np.array([z]) 
        z = np.asarray(z)
        segments = make_segments(x, y)
        lc = LineCollection(segments, array=z, cmap=cmap, norm=norm, linewidth=linewidth, alpha=alpha, zorder = 3)
        ax = plt.gca()
        ax.add_collection(lc)
        return lc

    ###################  Plot Trajectories in x-y plane ###############################

    #Create Plot
    ax = plt.subplot2grid((6, 10), (2, 0), rowspan=4, colspan = 8, aspect = 'equal')


    #Plot cloud
    plot = plt.contourf(qxy[t,:,:], levels, cmap = cmapf, extend = 'max', alpha = alpha,  zorder = 3, vmin = 0, antialiased = TF)



    # #Plot trajectories in x-y plane, colored by Variable1
    # #Plots trjectory with highest mean z first, then scond, then third...
    # zpos_seg = np.nanmean(zpos[:,:,:], axis = 0)
    # zord = np.argsort(zpos_seg[:,:])
    # for i in range(len(xpos[0,:,0])):
    #     for j in range(len(xpos[0,0,:])):
    #         ind  = zord[i, j]
    #         trajectories_xy = ax.scatter(xpos[t,i,ind], ypos[t,i,ind], variable1[t,i,ind], cmap = cmap, alpha = alpha_traj)

    trajectories_xy = ax.scatter(xpos[t,:,:], ypos[t,:,:],c = variable1[t,:,:], s = size, cmap = cmap, alpha = alpha_traj, zorder  = 10, norm=matplotlib.colors.BoundaryNorm(xlevels,cmap.N))

    #Ticks and labels (set by user)
    xticks = np.arange(0,ds.nx+1,400)
    yticks = np.arange(0,ds.ny+1,200)
    plt.xticks(xticks)
    plt.yticks(yticks)
    xtickslabs = (xticks*hor_resolution/1000).astype(int) #kilometers
    ytickslabs = (yticks*hor_resolution/1000).astype(int) #kilometers
    ax.set_xticklabels((xtickslabs), fontsize = 13)
    ax.set_yticklabels((ytickslabs), fontsize = 13)
    plt.ylabel('Distance (km)', fontsize = 17, labelpad = 0)
    plt.xlabel('Distance (km)', fontsize = 17)

    #Limits and characteristics (set by user)
    plt.xlim(left,right)
    plt.ylim(y_bot, y_top)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    #Plot water and land (set by user)
    levels_water = [1.5, 2.5]
    levels_terrain = [0,1.5]
    xland_plt = plt.contourf(ds.xland[0,:,:], levels_water, alpha = 1, colors = ('lightsteelblue'), zorder = 2)
    xland_plt = plt.contourf(ds.xland[0,:,:], levels_terrain, alpha = 1, colors = ('gainsboro'), zorder = 1)

    #Terrain (set by user)
    terrain_levels = np.arange(-1, 2000.1, 100)
    terrain = plt.contourf(zs[:,:], terrain_levels, alpha = 1, cmap = cm.Greys, vmin = -600, vmax = 2000, zorder = 1)

    #Label
    props = dict(boxstyle='square', facecolor='white', alpha=0.8, ec="gray")
    ax.text(0.02, 0.915, '(b) XY',\
            transform=ax.transAxes, bbox = props, fontsize = 16, zorder = 20)



    ###################  Plot Trajectories in x-z plane ###############################

    #Create subplot
    ax = plt.subplot2grid((6, 10), (0, 0), rowspan=2, colspan = 8)

    #Plot cloud
    plot_q = plt.contourf((x2d-x2d[0,0])*1000/125, z2dxz-50, qxz[t,:,:], levels, cmap = cmapf, extend = 'max', alpha = alpha,  zorder = 3, vmin = 0, antialiased = TF)



    # #Plot trajectories in x-y plane, colored by Variable1
    # #Plots trjectory with lowest mean y first, then scond, then third...
    # ypos_seg = np.nanmean(ypos[:,:,:], axis = 0)
    # yord = np.argsort(ypos_seg[:,:])
    # for i in range(len(xpos[0,:,0])):
    #     for j in range(len(xpos[0,0,:])):
    #         ind  = yord[-i, -j]
    #         trajectories_xz = colorline(xpos[:,i,ind], zpos[:,i,ind], variable1[:,i,ind], cmap, alpha = alpha_traj, linewidth = width)
    
    trajectories_xz = ax.scatter(xpos[t,:,:], zpos[t,:,:],c = variable1[t,:,:], s = size, cmap = cmap, alpha = alpha_traj, zorder  = 10, norm=matplotlib.colors.BoundaryNorm(xlevels,cmap.N))

    #Ticks and labels (set by user)
    xticks = np.arange(0,ds.nx+1,400)
    yticks = np.arange(0,4001,1000)
    plt.xticks(xticks)
    plt.yticks(yticks)
    xtickslabs = (xticks*hor_resolution/1000).astype(int) #kilometers
    ytickslabs = (yticks/1000).astype(int) #km
    ax.set_xticklabels((xtickslabs), fontsize = 13)
    ax.set_yticklabels((ytickslabs), fontsize = 13)
    plt.ylabel('Height (km)', fontsize = 17)

    #Limits and characteristics (set by user)
    plt.xlim(left,right)
    plt.ylim([-70,yticks[-1]])
    plt.axvspan(0,ds.nx,color='gainsboro',lw=0)
    plt.grid(True, color = 'white')

    #Plot Terrain (set by user)
    x1d = np.arange(0,ds.nx,1)
    ymid = np.int(ds.ny/2)
    terrain = plt.plot(x1d, zs[ymid,:]-40, c = 'slategrey', linewidth = 4, zorder = 4)

    #Plot Lake (set by user)
    lake = np.array(ds.xland[0,ymid,:])
    lake[lake == 1] = np.nan
    lake_plt = plt.plot(x1d, lake-40, c = 'blue', linewidth = 4, zorder = 5)

    #Label
    ax.text(0.02, 0.82, '(a) XZ',\
            transform=ax.transAxes, bbox = props, fontsize = 16, zorder = 20)

    ###################  Plot Trajectories in y-z plane ###############################

    #Create subplot
    ax = plt.subplot2grid((6, 10), (2, 8), rowspan=4, colspan = 2)



    #To make same lengtho as x-z plot
    scale = 3.6

    plot = plt.contourf(np.rot90(z2dyz,3)-50, np.rot90((y2d-y2d[0,0])*1000/125,3), np.rot90(qyz[t,:,:],3), levels, cmap = cmapf, extend = 'max', alpha = alpha,  zorder = 3, vmin = 0, antialiased = TF)


    # #Plots trjectory with lowest mean x first, then scond, then third...
    # xpos_seg = np.nanmean(xpos[:,:,:], axis = 0)
    # xord = np.argsort(xpos_seg[:,:])
    # for i in range(len(xpos[0,:,0])):
    #     for j in range(len(xpos[0,0,:])):
    #         ind  = xord[-i, -j]
    #         trajectories_xz = colorline(zpos[:,i,ind], ypos[:,i,ind], variable1[:,i,ind], cmap, alpha = alpha_traj, linewidth = width)
    
    trajectories_yz = ax.scatter(zpos[t,:,:], ypos[t,:,:],c = variable1[t,:,:], s = size, cmap = cmap, alpha = alpha_traj, zorder  = 10, norm=matplotlib.colors.BoundaryNorm(xlevels,cmap.N))

    #Ticks and labels (set by user)
    yticks = np.arange(0,ds.ny+1,200)
    xticks = np.arange(0,4001,1000)
    plt.yticks(yticks)
    plt.xticks(xticks)
    ytickslabs = (yticks*hor_resolution/1000).astype(int) #kilometers
    xtickslabs = (xticks/1000).astype(int) #km
    ax.set_yticklabels((ytickslabs), fontsize = 13)
    ax.set_xticklabels((xtickslabs), fontsize = 13)
    plt.xlabel('Height (km)', fontsize = 17)

    #Limits and characteristics (set by user)
    plt.ylim(y_bot,y_top)
    plt.xlim(-50,4000)
    plt.axvspan(0,4000,color='gainsboro',lw=0)
    plt.grid(True, color = 'white')


    #Plot Lake (set by user)
    y1d = np.arange(0,ds.ny,1)
    lake = np.array(ds.xland[0,:,2000])
    lake[lake == 1] = np.nan
    lake_plt = plt.plot(lake-15, y1d, c = 'blue', linewidth = 4, zorder = 5)

    #Plot Terrain (set by user)
    y1d = np.arange(0,ds.ny,1)
    terrain = plt.plot(zs[:,2000]-15, y1d, c = 'slategrey', linewidth = 4, zorder = 4)

    #Label
    ax.text(0.11, 0.92, '(c) YZ',\
            transform=ax.transAxes, bbox = props, fontsize = 16, zorder = 20)
    
    #Time Label
    ax.text(-5.4, -0.13, 'Elapsed time: {:,} seconds'.format(np.int(t*90)),\
            transform=ax.transAxes, fontsize = 14, zorder = 20)



    #Colorbar
    cbaxes = fig.add_axes([0.85, 0.74, 0.025, 0.23])
    cbar = plt.colorbar(trajectories_xy, orientation='vertical', cax = cbaxes, ticks = xlevels_ticks)
    cbar.ax.set_yticklabels(xlevels_ticks_labels)
    cbar.ax.tick_params(labelsize=12)
    plt.text(0.55,-0.15, 'K', fontsize = 16, ha='center', va='center')

    #Colorbar
    cbaxes = fig.add_axes([0.94, 0.74, 0.025, 0.23])
    cbar = plt.colorbar(plot_q, orientation='vertical', cax = cbaxes, ticks = levels_ticks)
    cbar.ax.set_yticklabels(levels_ticks_labels)
    cbar.ax.tick_params(labelsize=12)
    plt.text(0.5,-0.15, 'dBZ', fontsize = 16, ha='center', va='center')


    #Save
    print(t)
    plt.savefig("/uufs/chpc.utah.edu/common/home/u1013082/public_html/phd_plots/cm1/png_for_gifs/trajectory_plot_2000m_tug_forward_lowres_{:03d}.png".format(t), dpi = 70)
    plt.close()
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
### Run multiprocess function to create images
from multiprocessing import Pool

if __name__ == '__main__':

    p = Pool(5) #Need to change this number to rerun wihtout clearing kernal
    images = np.arange(20,23,1)
    for res in p.map(plotting,images):
        print(res)

OSError: [Errno 12] Cannot allocate memory